In [1]:

import pandas as pd
import requests, json

date = '20250201'
stock_no = '2317'

html = requests.get('https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=%s&stockNo=%s' % (date,stock_no))
content = json.loads(html.text)
stock_data = content['data']
col_name = content['fields']

df = pd.DataFrame(data=stock_data, columns=col_name)
df.head(20)

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,114/02/03,"168,103,694","27,880,932,586",164.00,168.00,163.50,165.50,-14.50,"203,537"
1,114/02/04,"68,458,788","11,379,696,528",167.00,169.00,165.00,165.00,-0.50,"73,349"
2,114/02/05,"53,304,973","9,033,062,898",167.00,171.50,167.00,171.00,+6.00,"42,917"
3,114/02/06,"41,060,652","7,081,097,296",172.50,174.00,171.00,173.00,+2.00,"31,971"
4,114/02/07,"52,118,210","9,202,320,428",175.00,178.00,174.00,178.00,+5.00,"44,998"
5,114/02/10,"37,723,196","6,590,892,944",176.00,176.50,174.00,174.00,-4.00,"48,491"
6,114/02/11,"71,048,641","12,782,953,142",178.00,182.00,176.00,179.00,+5.00,"48,761"
7,114/02/12,"42,666,219","7,608,185,758",181.50,181.50,177.00,177.00,-2.00,"63,003"
8,114/02/13,"40,596,733","7,259,602,358",177.00,181.00,177.00,180.00,+3.00,"24,848"
9,114/02/14,"26,925,038","4,808,438,797",180.00,180.50,177.50,178.00,-2.00,"25,657"


In [4]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta

def fetch_stock_data(stock_no, start_date, end_date):
    all_data = []

    # 逐月抓取數據
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")  # 轉換成 YYYYMMDD 格式
        url = f'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date={date_str}&stockNo={stock_no}'

        response = requests.get(url)
        if response.status_code == 200:
            content = json.loads(response.text)
            if 'data' in content and 'fields' in content:
                stock_data = content['data']
                col_name = content['fields']

                df = pd.DataFrame(data=stock_data, columns=col_name)
                all_data.append(df)

        # 移動到下一個月
        next_month = current_date.month % 12 + 1
        next_year = current_date.year + (1 if current_date.month == 12 else 0)
        current_date = datetime(next_year, next_month, 1)

    # 合併所有 DataFrame
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return None

# 設定查詢範圍
stock_no = '2454'
start_date = '20240101'
end_date = '20250131'

df = fetch_stock_data(stock_no, start_date, end_date)

if df is not None:
    print(df.head(20))  # 顯示前 20 筆資料
    df.to_csv(f"{stock_no}_{start_date}_{end_date}.csv", index=False)  # 存成 CSV 檔案
else:
    print("無法獲取股票數據")


           日期        成交股數            成交金額       開盤價       最高價     最低價     收盤價  \
0   113/01/02  10,668,557  10,539,569,604  1,010.00  1,015.00  980.00  981.00   
1   113/01/03  13,403,900  12,879,381,226    976.00    977.00  953.00  953.00   
2   113/01/04   9,628,243   8,961,976,323    940.00    947.00  922.00  928.00   
3   113/01/05   5,851,876   5,418,281,372    929.00    935.00  921.00  921.00   
4   113/01/08   6,035,931   5,581,861,138    930.00    936.00  918.00  920.00   
5   113/01/09   5,893,697   5,489,896,904    938.00    940.00  926.00  928.00   
6   113/01/10   4,380,442   4,054,858,477    935.00    935.00  922.00  924.00   
7   113/01/11   7,008,030   6,482,779,978    932.00    937.00  920.00  920.00   
8   113/01/12   4,936,513   4,561,920,934    920.00    929.00  916.00  926.00   
9   113/01/15   4,413,758   4,080,345,389    935.00    935.00  919.00  922.00   
10  113/01/16  11,367,477  10,254,923,229    915.00    917.00  895.00  895.00   
11  113/01/17  12,081,845  1